# Análisis de Texto

#### Importar librerias

In [1]:
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import nltk
import glob
import re
import urllib
import requests
import os
import wget
import time
from collections import defaultdict
from sklearn import preprocessing
from bs4 import BeautifulSoup
from urllib.request import urlopen
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

### Cargar archivo csv

In [2]:
pines = pd.read_csv("pins_20220306.csv")

In [3]:
pines.rename(columns={'pin_id':'id','comment':'comentarios','created_at':'creado','domain':'dominio'}, inplace=True)

In [4]:
pines.shape

(305740, 15)

#### Transformar a NaN espacios en blanco en columna [comentarios]

In [5]:
pines["comentarios"] = pines["comentarios"].replace(' ', np.nan, inplace=False)

#### Eliminar todas las filas con valor NaN en columna [comentarios]

In [6]:
pines = pines.dropna(axis=0, subset=["comentarios"])

In [7]:
pines.head(3)

,Unnamed: 0,Unnamed: 0.1,id,userId,userName,dominio,img_Url,comentarios,title,creado,follower_count,locale,reaction_counts,repin_count,Unnamed: 0.1.1
4,4,4,411305378457667982,1.071014e+17,oyshostore,oysho.com,https://i.pinimg.com/originals/e8/96/1d/e8961d...,Orchid flower nightdress - New In - LINGERIE |...,Orchid flower nightdress - New In - LINGERIE |...,"Fri, 21 Apr 2017 05:08:25 +0000",58163,es-ES,{},0,NaN
5,5,5,737534876445681460,1.071014e+17,oyshostore,oysho.com,https://i.pinimg.com/originals/9f/3f/94/9f3f94...,Grey striped rabbit top - New In - Autumn Wint...,Grey striped rabbit top - New In - NIGHTWEAR |...,"Sat, 01 Oct 2016 09:58:22 +0000",58163,es-ES,{},0,NaN
6,6,6,250372060513819486,1.071014e+17,oyshostore,oysho.com,https://i.pinimg.com/originals/4f/cd/28/4fcd28...,Jersey stripe trousers - New In - Autumn Winte...,Jersey stripe trousers - New In - ACTIVEWEAR |...,"Wed, 19 Oct 2016 07:53:38 +0000",58163,es-ES,{},0,NaN


In [8]:
pines.shape

(204700, 15)

### Delimitar temporada y el rango de años en columna [creado]

In [9]:
pines.creado

4         Fri, 21 Apr 2017 05:08:25 +0000
5         Sat, 01 Oct 2016 09:58:22 +0000
6         Wed, 19 Oct 2016 07:53:38 +0000
7         Tue, 20 Dec 2016 20:32:55 +0000
8         Wed, 30 Nov 2016 15:35:19 +0000
                       ...               
305731    Wed, 10 Nov 2021 08:09:12 +0000
305733    Mon, 27 Sep 2021 04:24:10 +0000
305735    Thu, 23 Sep 2021 05:43:16 +0000
305736    Mon, 27 Sep 2021 04:23:43 +0000
305738    Mon, 27 Sep 2021 04:08:21 +0000
Name: creado, Length: 204700, dtype: object

In [10]:
pines["creado"] = [i[5:-15] for i in pines.creado]

In [11]:
pines['creado'] = pd.to_datetime(pines['creado'])


##### Ordenar por año en ascendencia

In [12]:
pines = pines.sort_values(by=["creado"],ascending=True)

In [13]:
pines.head()

,Unnamed: 0,Unnamed: 0.1,id,userId,userName,dominio,img_Url,comentarios,title,creado,follower_count,locale,reaction_counts,repin_count,Unnamed: 0.1.1
115964,3090,3090,46795283597629927,1.719814e+17,metmuseum,metmuseum.org,https://i.pinimg.com/originals/7e/2d/8d/7e2d8d...,love the lines on this swimming suit,Rudi Gernreich | Bikini | American | The Metro...,2010-11-18,887277,en-US,{},2,NaN
268834,115964,3090,46795283597629927,1.719814e+17,metmuseum,metmuseum.org,https://i.pinimg.com/originals/7e/2d/8d/7e2d8d...,love the lines on this swimming suit,Rudi Gernreich | Bikini | American | The Metro...,2010-11-18,887277,en-US,{},2,3090.0
165656,12786,2786,211174952011305,2.113124e+14,https://www.pinterest.com/duchampian/,Uploaded by user,https://i.pinimg.com/originals/79/6d/07/796d07...,new bathing suit. just had to have it. wish it...,NaN,2011-03-24,0,NaN,{},0,2786.0
12786,2786,2786,211174952011305,2.113124e+14,https://www.pinterest.com/duchampian/,Uploaded by user,https://i.pinimg.com/originals/79/6d/07/796d07...,new bathing suit. just had to have it. wish it...,NaN,2011-03-24,0,NaN,{},0,NaN
33082,3082,3082,37858453087064062,3.785859e+16,https://www.pinterest.com/iforgotyourface/,glamoursplash.com,https://i.pinimg.com/originals/8d/21/f7/8d21f7...,THESE COLORS. THAT BACK HARNESS.,As seen from the back - Vintage Swimwear 1961,2011-05-17,0,NaN,{},1,NaN


#### Rango (2015 - 2020) entre meses (Marzo - Septiembre) cada año

In [14]:
pines_filtrados = pines[pines["creado"].isin(pd.date_range('2016-03-01', '2020-09-30'))]

In [15]:
pines_filtrados = pines_filtrados[(pines_filtrados['creado'].dt.strftime('%m') >= '03') & (pines['creado'].dt.strftime('%m') <= '09')]


/Users/JuanLeyva/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


##### Detectar si hay comentarios en meses posteriores a Septiembre

In [16]:
pines_filtrados[pines_filtrados['creado'].dt.strftime('%m') > '09']


,Unnamed: 0,Unnamed: 0.1,id,userId,userName,dominio,img_Url,comentarios,title,creado,follower_count,locale,reaction_counts,repin_count,Unnamed: 0.1.1


In [17]:
pines_filtrados.shape

(70388, 15)

### Función limpieza & conversion a Tokens

In [18]:
def unigram_tokens(comentarios): 
    # Se convierte todo el texto a minúsculas
    comentario = comentarios.lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    comentario = re.sub('http\S+', ' ', comentario)
    # Eliminación de hipervinculos
    comentario = re.sub('href\S+', ' ', comentario)
    # Eliminación de signos de puntuación
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\]\\^_\\`\\{\\|\\}\\~]'
    comentario = re.sub(regex , ' ', comentario)
    # Eliminación de espacios en blanco múltiples
    comentario = re.sub("\\s+", ' ', comentario)
    # Tokenización por palabras individuales
    comentario = comentario.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    comentario = [token for token in comentario if len(token) > 1]
    # Eliminacion de las StopWords
    stop_words = set(stopwords.words('english'))
    comentario = [w for w in comentario if not w.lower() in stop_words]
    
    return(comentario)

## N-grams

#### Unigram

##### Tokenizacion por comentario

In [19]:
pines_filtrados['unigram_tokens'] = pines_filtrados['comentarios'].apply(lambda x: unigram_tokens(x))
pines_filtrados[['comentarios', 'unigram_tokens']].head()

,comentarios,unigram_tokens
626,Seea Adria Bikini Top,"[seea, adria, bikini, top]"
186566,Out From Under Carolina Maxi Slip,"[carolina, maxi, slip]"
158268,Dsquared2 - Spring 2016 RTW - The Cut,"[dsquared2, spring, 2016, rtw, cut]"
66158,Monokini crochet,"[monokini, crochet]"
221445,Anthropologie,[anthropologie]


#### Bigrams

##### Tokenizacion por comentario

In [20]:
pines_filtrados['bigram_tokens'] = pines_filtrados['unigram_tokens'].apply(lambda x: list(ngrams(x,2)))

In [21]:
pines_filtrados['bigram_tokens'] = pines_filtrados['bigram_tokens'].apply(lambda x: [*map(' '.join,x)])

In [22]:
pines_filtrados['unigram_tokens'].head()

626                [seea, adria, bikini, top]
186566                 [carolina, maxi, slip]
158268    [dsquared2, spring, 2016, rtw, cut]
66158                     [monokini, crochet]
221445                        [anthropologie]
Name: unigram_tokens, dtype: object

In [23]:
pines_filtrados['bigram_tokens'].head()

626                  [seea adria, adria bikini, bikini top]
186566                           [carolina maxi, maxi slip]
158268    [dsquared2 spring, spring 2016, 2016 rtw, rtw ...
66158                                    [monokini crochet]
221445                                                   []
Name: bigram_tokens, dtype: object

### Leer archivo xlsx - archivo(Catart)

In [24]:
cat = pd.read_excel("Catart_2.xlsx")

#### Preprocesamiento y limpieza - archivo(Catart)

##### Guardar cada atributo en una variable

In [25]:
color = cat.COLOR
estilo = cat.ESTILO
tela = cat.DESCCOMPOSICION

##### Eliminar valores NaN

In [26]:
color = color.dropna()
estilo = estilo.dropna()
tela = tela.dropna()

##### Convertir todos los elementos en minusculas

In [27]:
color = color.str.lower()
estilo = estilo.str.lower()
tela = tela.str.lower()

##### Eliminar elementos duplicados

In [28]:
color = color.drop_duplicates()
estilo = estilo.drop_duplicates()
tela = tela.drop_duplicates()

##### Seleccionar solo los elementos de estilo

In [29]:
#estilo = [i[3:] for i in estilo]


##### Convertir en lista cada atributo

In [30]:
estilo = list(dict.fromkeys(estilo))

In [31]:
tela = list(dict.fromkeys(tela))

In [32]:
color = list(dict.fromkeys(color))

## Nested List Comprenhensions

#### Detectar atributos (color, tela y estilo) por comentario - (Datos entrenamiento)

In [33]:
pines_filtrados['bigram_tokens']

626                  [seea adria, adria bikini, bikini top]
186566                           [carolina maxi, maxi slip]
158268    [dsquared2 spring, spring 2016, 2016 rtw, rtw ...
66158                                    [monokini crochet]
221445                                                   []
                                ...                        
121905    [new acacia, acacia orchid, orchid crochet, cr...
160932    [shop fast, fast growing, growing online, onli...
107040    [33 white, white jumpsuits, jumpsuits weddings...
137678    [first shop, shop new, new arrivals, arrivals ...
276779    [whether you’re, you’re planning, planning nex...
Name: bigram_tokens, Length: 70388, dtype: object

In [34]:
Atributo_C = []
for comentario in pines_filtrados['unigram_tokens']:
    atributo = [token for token in comentario if token in color if token in comentario]
    Atributo_C.append(atributo)


In [35]:
Atributo_T = []
for comentario in pines_filtrados['bigram_tokens']:
    atributo = [elemento for subs in comentario for elemento in tela if elemento in subs]
    Atributo_T.append(atributo)


In [36]:
Atributo_E = []
for comentario in pines_filtrados['bigram_tokens']:
    atributo = [elemento for subs in comentario for elemento in estilo if elemento in subs]
    Atributo_E.append(atributo)


##### Convertir atributos en Dataframe

In [37]:
df_c = pd.DataFrame(Atributo_C)
df_t = pd.DataFrame(Atributo_T)
df_e = pd.DataFrame(Atributo_E)

##### Renombrar columnas en Dataframe

In [38]:
df_c.rename(columns = {0 : "color_1", 1: "color_2", 2: "color_3"},inplace=True)
df_t.rename(columns = {0 : "tela_1", 1: "tela_2", 2: "tela_3"},inplace=True)
df_e.rename(columns = {0 : "estilo_1", 1: "estilo_2", 2: "estilo_3"},inplace=True)

##### Seleccionar columnas especificas en Dataframe

In [39]:
df_c = df_c[["color_1", "color_2", "color_3"]]
df_t = df_t[["tela_1", "tela_2", "tela_3"]]
df_e = df_e[["estilo_1", "estilo_2", "estilo_3"]]

### Dataframe con columnas de interes

##### Eliminar columnas sin relevancia

In [40]:
df = pines_filtrados.drop(columns=["Unnamed: 0","Unnamed: 0.1","img_Url","title","unigram_tokens","bigram_tokens","reaction_counts","locale","Unnamed: 0.1.1"], axis= 1).reset_index()

##### Concatenar Dataframes para un analisis posterior

In [41]:
df = pd.concat([df, df_c, df_t, df_e], axis=1, ignore_index=True)

In [42]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,626,313140980321868905,2.598016e+17,urbanoutfitters,urbanoutfitters.com,Seea Adria Bikini Top,2016-03-01,3059122,1,None,None,None,None,None,None,None,None,None
1,186566,70791025371281551,2.598016e+17,urbanoutfitters,urbanoutfitters.com,Out From Under Carolina Maxi Slip,2016-03-01,3059125,0,None,None,None,None,None,None,None,None,None
2,158268,465700417698566187,4.517694e+16,nymag,nymag.com,Dsquared2 - Spring 2016 RTW - The Cut,2016-03-01,180274,0,None,None,None,None,None,None,None,None,None
3,66158,407857309985061022,3.269334e+17,weloveboho,weloveboho.com,Monokini crochet,2016-03-01,201442,0,None,None,None,None,None,None,None,None,None
4,221445,307511480788033140,2.427742e+16,anthropologie,anthropologie.com,Anthropologie,2016-03-01,1605335,0,None,None,None,None,None,None,None,None,None


##### Renombrar columnas del Dataframe

In [43]:
df = df.rename(columns={0:"index",1:"id",2:"usuario_id",3:"usuario",4:"dominio",5:"comentarios",6:"creado",7:"Num_seguidores",8:"repin_count",9:"atributo_color_1",10:"atributo_color_2",
                        11:"atributo_color_3",12:"atributo_tela_1",13:"atributo_tela_2",14:"atributo_tela_3",15:"atributo_estilo_1",16:"atributo_estilo_2",17:"atributo_estilo_3"})


##### Conversion a string - (columna [usuario_id])

In [44]:
df['usuario_id'] = df['usuario_id'].apply(lambda x: '{:.0f}'.format(x))
df['usuario_id'] = df['usuario_id'].astype(str)

##### Eliminar pines duplicados

In [45]:
df = df.drop_duplicates(subset=['id'])

### Dataframe con atributos (max 3 elementos por atributo)

In [46]:
df_atributos = df[["id","usuario_id","usuario","comentarios","creado","Num_seguidores","repin_count","atributo_color_1","atributo_color_2","atributo_color_3","atributo_tela_1","atributo_tela_2","atributo_tela_3","atributo_estilo_1","atributo_estilo_2","atributo_estilo_3"]]


In [47]:
df_atributos.head()

,id,usuario_id,usuario,comentarios,creado,Num_seguidores,repin_count,atributo_color_1,atributo_color_2,atributo_color_3,atributo_tela_1,atributo_tela_2,atributo_tela_3,atributo_estilo_1,atributo_estilo_2,atributo_estilo_3
0,313140980321868905,259801609662367008,urbanoutfitters,Seea Adria Bikini Top,2016-03-01,3059122,1,None,None,None,None,None,None,None,None,None
1,70791025371281551,259801609662367008,urbanoutfitters,Out From Under Carolina Maxi Slip,2016-03-01,3059125,0,None,None,None,None,None,None,None,None,None
2,465700417698566187,45176939920493432,nymag,Dsquared2 - Spring 2016 RTW - The Cut,2016-03-01,180274,0,None,None,None,None,None,None,None,None,None
3,407857309985061022,326933391607916864,weloveboho,Monokini crochet,2016-03-01,201442,0,None,None,None,None,None,None,None,None,None
4,307511480788033140,24277422899724312,anthropologie,Anthropologie,2016-03-01,1605335,0,None,None,None,None,None,None,None,None,None


##### Reemplazar los valores "None" de multiples columnas seleccionadas por 0

In [48]:
df_atributos[["atributo_color_1","atributo_color_2","atributo_color_3","atributo_tela_1","atributo_tela_2","atributo_tela_3","atributo_estilo_1","atributo_estilo_2","atributo_estilo_3"]] = df_atributos[["atributo_color_1","atributo_color_2","atributo_color_3","atributo_tela_1","atributo_tela_2","atributo_tela_3","atributo_estilo_1","atributo_estilo_2","atributo_estilo_3"]].fillna(0)


/Users/JuanLeyva/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


##### Delimitar a solo pines que contengan al menos un atributo (color, tela y estilo) distinto a 0

In [49]:
df_atributos = df_atributos.loc[(df_atributos.atributo_color_1 != 0) | (df_atributos.atributo_tela_1 != 0) | (df_atributos.atributo_estilo_1 != 0)].reset_index()


##### Etiqueta de atributos (color, tela y estilo)

In [50]:
df_atributos["color"] = 0
df_atributos["tela"] = 0
df_atributos["estilo"] = 0

##### Eliminar columna [index]

In [51]:
df_atributos = df_atributos.drop(columns = ["index"])

In [52]:
df_atributos

,id,usuario_id,usuario,comentarios,creado,Num_seguidores,repin_count,atributo_color_1,atributo_color_2,atributo_color_3,atributo_tela_1,atributo_tela_2,atributo_tela_3,atributo_estilo_1,atributo_estilo_2,atributo_estilo_3,color,tela,estilo
0,129689664247791019,129689801677865344,POPSUGARFitness,"With full coverage and a shelf bra, you'll be ...",2016-03-01,748646,13,0,0,0,0,0,0,one piece,0,0,0,0,0
1,492651646718979160,465559817637892800,DressTrendscom,sport-coat-with-jeans-4,2016-03-01,2781,1,0,0,0,0,0,0,sport,0,0,0,0,0
2,223420831492840339,137289626049056160,target,Women's Push Up Cut-Out Bandeau One Piece - Xh...,2016-03-01,4287730,0,0,0,0,0,0,0,one piece,0,0,0,0,0
3,238339005257824737,238339142688181504,https://www.pinterest.com/acfr1968/,VIX Agatha embroidered voile maxi dress | Cut ...,2016-03-01,0,0,0,0,0,voile,voile,voile,0,0,0,0,0,0
4,43558321373797279,43558458804407040,https://www.pinterest.com/madvintage/,Super fun 1970s romper set! Mini romper top/dr...,2016-03-01,0,0,royal,blue,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14509,842313936552154067,560416884789374464,rockindocks,Material:PolyesterStyle:FashionSleeve Length(c...,2020-09-29,8149,0,0,0,0,polyester,poly,polyester,normal,normal,0,0,0,0
14510,704602304195741431,699113679563577984,ledikacani,GONNA DI TULLE AUTUNNO INVERNO 2018/2019,2020-09-29,61,0,0,0,0,tul,tul,0,0,0,0,0,0,0
14511,56224695337645650,23432997969621924,theposhmarkapp,New acacia orchid crochet panama bikini top sw...,2020-09-30,683462,0,pink,purple,0,0,0,0,0,0,0,0,0,0
14512,98094098123977732,121104814888194720,seokingz,Sexy V-neck One-piece Swimsuit Women high wais...,2020-09-30,10785,3,black,white,black,0,0,0,one piece,thong,thong,0,0,0


##### Exportar dataframe a archivo csv

In [53]:
df_atributos.to_csv('df_atributos.csv')

### Dataframe con atributos etiquetados por su valor en diccionario

In [54]:
df_atributos_tag = df[["id","usuario_id","usuario","creado","Num_seguidores","repin_count","atributo_color_1","atributo_color_2","atributo_color_3","atributo_tela_1","atributo_tela_2","atributo_tela_3","atributo_estilo_1","atributo_estilo_2","atributo_estilo_3"]]


In [55]:
df_atributos_tag.head()

,id,usuario_id,usuario,creado,Num_seguidores,repin_count,atributo_color_1,atributo_color_2,atributo_color_3,atributo_tela_1,atributo_tela_2,atributo_tela_3,atributo_estilo_1,atributo_estilo_2,atributo_estilo_3
0,313140980321868905,259801609662367008,urbanoutfitters,2016-03-01,3059122,1,None,None,None,None,None,None,None,None,None
1,70791025371281551,259801609662367008,urbanoutfitters,2016-03-01,3059125,0,None,None,None,None,None,None,None,None,None
2,465700417698566187,45176939920493432,nymag,2016-03-01,180274,0,None,None,None,None,None,None,None,None,None
3,407857309985061022,326933391607916864,weloveboho,2016-03-01,201442,0,None,None,None,None,None,None,None,None,None
4,307511480788033140,24277422899724312,anthropologie,2016-03-01,1605335,0,None,None,None,None,None,None,None,None,None


#### Dictionary comprenhension (color)

Insertar valor "None" en lista color

In [56]:
color.insert(0, None)

In [57]:
color_dict = {k: v for v, k in enumerate(color)}

Reemplazar elemento de atributo(color) en multiples columnas seleccionadas por su valor en diccionario

In [58]:
df_atributos_tag = df_atributos_tag.replace({"atributo_color_1":color_dict,"atributo_color_2":color_dict,"atributo_color_3":color_dict})


#### Dictionary comprenhension (tela)

Insertar valor "None" en lista tela

In [59]:
tela.insert(0, None)

In [60]:
tela_dict = {k: v for v, k in enumerate(tela)}

Reemplazar elemento de atributo(tela) de multiples columnas seleccionadas por su valor en diccionario

In [61]:
df_atributos_tag = df_atributos_tag.replace({"atributo_tela_1":tela_dict,"atributo_tela_2":tela_dict,"atributo_tela_3":tela_dict})


#### Dictionary comprenhension (estilo)

Insertar valor "None" en lista estilo

In [62]:
estilo.insert(0, None)

In [63]:
estilo_dict = {k: v for v, k in enumerate(estilo)}

Reemplazar elemento de atributo(estilo) en multiples columnas seleccionadas por su valor en diccionario

In [64]:
df_atributos_tag = df_atributos_tag.replace({"atributo_estilo_1":estilo_dict,"atributo_estilo_2":estilo_dict,"atributo_estilo_3":estilo_dict})


#### Dataframe con pines que tienen detectan al menos un atributo - dimension (15403 filas, 15 columnas)

##### Delimitar a solo pines que contengan al menos un atributo (color, tela y estilo) distinto a 0

In [65]:
df_atributos_tag = df_atributos_tag[(df_atributos_tag.atributo_color_1 != 0) | (df_atributos_tag.atributo_tela_1 != 0) | (df_atributos_tag.atributo_estilo_1 != 0)].reset_index()


##### Eliminar columna [index]

In [66]:
df_atributos_tag = df_atributos_tag.drop(columns = ["index"])

In [67]:
df_atributos_tag

,id,usuario_id,usuario,creado,Num_seguidores,repin_count,atributo_color_1,atributo_color_2,atributo_color_3,atributo_tela_1,atributo_tela_2,atributo_tela_3,atributo_estilo_1,atributo_estilo_2,atributo_estilo_3
0,129689664247791019,129689801677865344,POPSUGARFitness,2016-03-01,748646,13,0,0,0,0,0,0,5,0,0
1,492651646718979160,465559817637892800,DressTrendscom,2016-03-01,2781,1,0,0,0,0,0,0,2,0,0
2,223420831492840339,137289626049056160,target,2016-03-01,4287730,0,0,0,0,0,0,0,5,0,0
3,238339005257824737,238339142688181504,https://www.pinterest.com/acfr1968/,2016-03-01,0,0,0,0,0,21,21,21,0,0,0
4,43558321373797279,43558458804407040,https://www.pinterest.com/madvintage/,2016-03-01,0,0,55,12,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14509,842313936552154067,560416884789374464,rockindocks,2020-09-29,8149,0,0,0,0,3,26,3,9,9,0
14510,704602304195741431,699113679563577984,ledikacani,2020-09-29,61,0,0,0,0,19,19,0,0,0,0
14511,56224695337645650,23432997969621924,theposhmarkapp,2020-09-30,683462,0,10,43,0,0,0,0,0,0,0
14512,98094098123977732,121104814888194720,seokingz,2020-09-30,10785,3,4,31,4,0,0,0,5,10,10


##### Exportar dataframe a archivo csv

In [68]:
df_atributos_tag.to_csv('df_atributos_tag.csv')

## Dataframe con atributos etiquetados de forma binaria

In [77]:
df_atributos_bin = df[["id","usuario_id","creado","Num_seguidores","atributo_color_1","atributo_color_2","atributo_color_3","atributo_tela_1","atributo_tela_2","atributo_tela_3","atributo_estilo_1","atributo_estilo_2","atributo_estilo_3"]]



In [78]:
df_atributos_bin.head()

,id,usuario_id,creado,Num_seguidores,atributo_color_1,atributo_color_2,atributo_color_3,atributo_tela_1,atributo_tela_2,atributo_tela_3,atributo_estilo_1,atributo_estilo_2,atributo_estilo_3
0,313140980321868905,259801609662367008,2016-03-01,3059122,None,None,None,None,None,None,None,None,None
1,70791025371281551,259801609662367008,2016-03-01,3059125,None,None,None,None,None,None,None,None,None
2,465700417698566187,45176939920493432,2016-03-01,180274,None,None,None,None,None,None,None,None,None
3,407857309985061022,326933391607916864,2016-03-01,201442,None,None,None,None,None,None,None,None,None
4,307511480788033140,24277422899724312,2016-03-01,1605335,None,None,None,None,None,None,None,None,None


Remover valor "None" de la respectiva lista de cada atributo

In [79]:
color.remove(None)
tela.remove(None)
estilo.remove(None)

ValueError: list.remove(x): x not in list

#### Etiquetado binario a base de condicion por atributo (color, tela y estilo)

El número maximo que se tiene de elementos por atributo es de 3 cada uno

In [80]:
df_atributos_bin[["atributo_color_1","atributo_color_2","atributo_color_3"]] = np.where(df_atributos_bin[["atributo_color_1","atributo_color_2","atributo_color_3"]].isin(color), '1','0')
df_atributos_bin[["atributo_tela_1","atributo_tela_2","atributo_tela_3"]] = np.where(df_atributos_bin[["atributo_tela_1","atributo_tela_2","atributo_tela_3"]].isin(tela), '1','0')
df_atributos_bin[["atributo_estilo_1","atributo_estilo_2","atributo_estilo_3"]] = np.where(df_atributos_bin[["atributo_estilo_1","atributo_estilo_2","atributo_estilo_3"]].isin(estilo), '1','0')

/Users/JuanLeyva/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


#### Dataframe con pines que tienen detectan al menos un atributo - dimension (20538 filas, 13 columnas)

##### Delimitar a solo pines que contengan al menos un atributo (color, tela y estilo) igual a 1

In [81]:
df_atributos_bin = df_atributos_bin[(df_atributos_bin.atributo_color_1 == "1") | (df_atributos_bin.atributo_tela_1 == "1") | (df_atributos_bin.atributo_estilo_1 == "1")].reset_index()


##### Eliminar columna [index]

In [82]:
df_atributos_bin = df_atributos_bin.drop(columns = ["index"])

In [83]:
df_atributos_bin

,id,usuario_id,creado,Num_seguidores,atributo_color_1,atributo_color_2,atributo_color_3,atributo_tela_1,atributo_tela_2,atributo_tela_3,atributo_estilo_1,atributo_estilo_2,atributo_estilo_3
0,129689664247791019,129689801677865344,2016-03-01,748646,0,0,0,0,0,0,1,0,0
1,492651646718979160,465559817637892800,2016-03-01,2781,0,0,0,0,0,0,1,0,0
2,223420831492840339,137289626049056160,2016-03-01,4287730,0,0,0,0,0,0,1,0,0
3,238339005257824737,238339142688181504,2016-03-01,0,0,0,0,1,1,1,0,0,0
4,43558321373797279,43558458804407040,2016-03-01,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14509,842313936552154067,560416884789374464,2020-09-29,8149,0,0,0,1,1,1,1,1,0
14510,704602304195741431,699113679563577984,2020-09-29,61,0,0,0,1,1,0,0,0,0
14511,56224695337645650,23432997969621924,2020-09-30,683462,1,1,0,0,0,0,0,0,0
14512,98094098123977732,121104814888194720,2020-09-30,10785,1,1,1,0,0,0,1,1,1


##### Exportar dataframe a archivo csv

In [84]:
df_atributos_bin.to_csv('df_atributos_bin.csv')